## Machine Learning for Neuroscience, <br>Department of Brain Sciences, Faculty of Medicine, <br> Imperial College London
### Contributors: Francesca Palermo, Nan Fletcher-Lloyd, Alex Capstick, Yu Chen
**Winter 2022**

# Neural Networks

This tutorial will focus on Neural Networks, and will rely on [scikit-learn's](https://scikit-learn.org/stable/index.html) and [pytorch's](https://pytorch.org) python packages.

In [ ]:
# importing packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import DecisionBoundaryDisplay
import sklearn.datasets as datasets
import numpy as np
import pandas as pd
import tqdm

# import plotting functions
import matplotlib.pyplot as plt
import matplotlib.colors
from cycler import cycler
binary_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#332288', 'white', '#AA4499'])
plt.rcParams["axes.prop_cycle"] = cycler(
    color=['#332288','#88CCEE','#44AA99','#117733','#999933','#DDCC77','#CC6677','#882255','#AA4499']
    )

# class for holding the random state throughout the notebook.
# this keeps results consistent
class RandomState(object):
    def __init__(self, random_state=None):
        self.random_state = random_state
    def next(self, n=1):
        assert type(n) == int, "Ensure n is an integer"
        if n == 1:
            self.random_state,\
                out_state = np.random.default_rng(
                    self.random_state
                    ).integers(0, 1e9, size=(2,))
        else:
            self.random_state,\
                *out_state = np.random.default_rng(
                    self.random_state
                    ).integers(0, 1e9, size=(n+1,))
        
        return out_state

In [ ]:
random_state = RandomState(42)

In [ ]:
torch.manual_seed(random_state.next())

## What is `torch`?


Pytorch is a python package that allows you to build machine learning models. It is highly flexible and allows you to perform auto-differentiation ✨!

All weights, inputs and targets in pytorch are stored as `torch.Tensor`s, ([documentation](https://pytorch.org/docs/stable/tensors.html)) which are very similar to numpy arrays, in the sense that they hold values but they hold the magic of being differentiable automatically. This makes training neural networks possible.

All neural network layers and loss functions are built using `nn.Module`s ([documentation](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)) which allow for differentiable manipulations of  `torch.Tensor`s.

## How to Train a Model

The basic pipeline for training a model is as follows:

<img src= "_dependents/pipeline.png" width="600" />

## The Building Blocks

To start, we will discuss the building blocks that go into a deep learning network. This will include how to access single layers, as well as the different types of activation functions that can be used to create non-linearity in models.

### A Single Neuron

The single neuron underpins much of deep learning, and is used in almost all deep learning models. In its most basic form, it is a vector of values that trasforms an incoming vector of data to a scalar value.

### The Basics

We can create this single neuron easily with pytorch's `torch.nn.Linear` class:

In [ ]:
# neuron that takes in 100 features and outputs 1 feature
sn = nn.Linear(10, 1, bias=False)

This is of the following form:

In [ ]:
sn.weight

Given an input, this class will produce the output by multiplying each of the feature values with the values in the weights. Let us see what this means:

In [ ]:
input = torch.rand(
    size=(1,10), # 1 data point with 10 features
    requires_grad=False, # data point should not be updated
    )

Now, we transform the input using our single neuron:

In [ ]:
sn(input)

Note that this output contains a `grad_fn`. This is pytorch's auto-grad system working in the background, ready to calculate the gradients to update weights in a model. We will see how this is used later.

Now, let's say that we want to have 5 neurons stacked on top of each other to transform a feature vector of size 10, to a feature vectore of size 5. This can easily be done in the following way:


In [ ]:
# 5 neurons acting together on the same input
fiven = nn.Linear(10, 5, bias=False)

In [ ]:
# given an input
input = torch.rand(
    size=(1,10), # 1 data point with 10 features
    requires_grad=False, # data point should not be updated
    )

# we can get the output
fiven(input)

This is the result of 5 neurons, each outputting a scalar value!

Pytorch allows you to stack these neurons vertically (acting on the same input) and horizontally (acting on the outputs from previous neurons). This will be discussed in the section [here](#a-feed-forward-network).

### Can We Train a Single Neuron?

Can we attempt to train this single neuron to replicate the equation: `y=mx+c`, for some `m` and `c`:

In [ ]:
m = torch.randn((1, 2), requires_grad=False) # a random m of size (1,2)
c = torch.randn(1, requires_grad=False) # a random c of size (1,)

print(f'The equation we are trying to predict is:\n y={np.round(m,2)}@x+{np.round(c,2)}')

In [ ]:
# 10000 train random inputs between 0 and 1 with 5 features 
X_train = torch.rand(size=(10000, 2), requires_grad=False)
# 10000 train random inputs between 0 and 1 with 5 features 
X_val = torch.rand(size=(1000, 2), requires_grad=False)
# normalise the data
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train)).float()
X_val = torch.tensor(scaler.transform(X_train)).float()

# with our set equation, this gives the y values
y_train = X_train@m.T + c
y_val = X_val@m.T + c

print(f'The y outputs are of shape {y_train.shape}')

We now need to build our single neuron model:

In [ ]:
# since here bias=True, this single neuron will take care of the m and c at the same time
sn = nn.Linear(2,1, bias=True)

In [ ]:
print(f"To start with, the single neuron has weights:\n {sn.weight} "\
    f"\n \n and bias:\n {sn.bias}")

To train this model, we will pass the `X` through the model and see how close it was to the true `y`.

In [ ]:
print(f"The first 5 predictions are:\n {sn(X_train)[:5]}"\
    f"\n The first 5 true values are:\n {y_train[:5]}")

At the moment this doesn't replicate the original function very well. We will therefore train this model using gradient descent to get it to perform better.

Now we can train this model (and by extension any model) using the following code:

For ease of training later in this notebook, the following code snippet will be added to the file `model_trainer.py` in this directory.

In [ ]:
# define training function that can be used 
# with any model, loss function, data and optimiser
def train(
    model, train_loader, n_epochs, optimiser, criterion, val_loader=None
    ):
    '''
    A function to train any model with a given dataset, optimiser, and 
    criterion (loss function).

    Arguments
    ---------
    
    - model: pytorch nn object:
        The model to train
    - train_loader: pytorch data loader:
        The data to train with.
    - n_epochs: integer:
        The number of epochs to train for.
    - optimiser: pytorch optimiser:
        The optimiser to make the model updates.
    - criterion: pytorch nn object:
        The loss function to calculate the loss with.
    - val_loader: pytorch data loader:
        The data to calculate the validation loss with.
    
    Returns
    ---------
    
    - model: pytorch nn object:
        Trained version of the model given
        as an input.
    - tuple of dictionaries:
        - train_loss_dict: dictionary:
            Dictionary containing the training loss
            with keys: `steps` and `loss`.
        - val_loss_dict
            Dictionary containing the validation loss
            with keys: `steps` and `loss`.

    '''
    # check if GPU is available and use that if so
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # ==== push model to GPU if available ====
    model.to(device)

    # since all functions in this function rely on the inputs above, 
    # they won't work outside of the train function

    # pass a single batch of data through the model and get loss
    def batch_loss(inputs, targets):
        # ==== push data to GPU if available ====
        inputs, targets = inputs.to(device), targets.to(device)
        # ==== forward pass ====
        outputs = model(inputs)
        # ==== calc and save loss ====
        loss = criterion(outputs, targets)
        # ==== return loss ====
        return loss
    
    # train for an epoch
    def train_epoch(train_loader):
        model.train() # set model option to train - important if using dropout
        batch_loss_list = [] # we will store all losses in a list
        # for each batch in the train loader
        for nb, (inputs, targets) in enumerate(train_loader):
            # ==== set gradient to zero ====
            optimiser.zero_grad() # really important! Common mistake to not do this!
            # run data through batch to get loss
            loss = batch_loss(inputs=inputs, targets=targets)
            # ==== calc backprop gradients ====
            loss.backward()
            # ==== perform update step ====
            optimiser.step()
            # ==== store loss for later ====
            batch_loss_list.append(loss.item())
        # ==== calculate average loss ====
        return batch_loss_list

    # perform an epoch over the validation data to get loss
    # dont want gradients in validation since we're not training
    @torch.no_grad()
    def val_epoch(val_loader):
        model.eval() # set model option to eval - important if using dropout
        batch_loss_list = [] # we will store all losses in a list
        # for each batch in the val loader
        for nb, (inputs, targets) in enumerate(val_loader):   
            # ==== set gradient to zero ====
            optimiser.zero_grad() # gradients shouldnt be calculated but good practise 
            # run data through batch to get loss
            loss = batch_loss(inputs=inputs, targets=targets)
            # ==== store loss for later ====
            batch_loss_list.append(loss.item())
        # ==== calculate average loss ====
        return batch_loss_list


    pbar = tqdm.tqdm(desc='Training', total=n_epochs) # progress bar
    # loss stats
    train_loss_dict = {'step': [], 'loss': []}
    val_loss_dict = {'step': [], 'loss': []}

    # train for the given n_epochs
    for ne in range(n_epochs):
        # ==== train for an epoch ====
        n_batches = len(train_loader)
        batch_lost_list_train = train_epoch(train_loader=train_loader)
        # ==== get loss stats ====
        train_loss_dict['loss'].extend(batch_lost_list_train) # adding loss
        # adding step values. These are the number of steps from the beginning
        train_loss_dict['step'].extend(
            list(np.arange(ne*n_batches, (ne+1)*n_batches)+1) 
            )
        avg_loss_train = np.mean(batch_lost_list_train)

        # if a validation loader is passed
        if val_loader is not None:  
            # ==== epoch over validation ====
            batch_lost_list_val = val_epoch(val_loader=val_loader)
            # ==== get loss stats ====
            avg_loss_val = np.mean(batch_lost_list_val)
            val_loss_dict['loss'].append(avg_loss_val)
            val_loss_dict['step'].append(
                (ne+1)*n_batches+1 # the number of new steps is as many as the train loader
                )
        else:
            avg_loss_val = np.nan

        # ==== set pbar info and update ====
        pbar.set_postfix(
            {
                'Train Loss': f"{avg_loss_train:.3f}",
                'Val Loss': f"{avg_loss_val:.3f}"
                }
            )
        pbar.update(1)
        pbar.refresh()
    
    # put the model back on the cpu
    model.to('cpu')

    return model, (train_loss_dict, val_loss_dict)

We then set the settings for the training, including the number of epochs

In [ ]:
# the number of times to run through the whole dataset
n_epochs = 10
# the batch size for the data
batch_size = 512

# wrapper data in dataset
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)

# put dataset in train loader
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    )
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True,
    )

To do this, we need a loss function and an optimiser:

In [ ]:
# pytorch has lots of optimisers available for use. Here we use Adam:
optimiser = torch.optim.SGD(
    params=sn.parameters(), # need to pass the optimiser the model parameters
    lr=0.01 # and the learning rate
    )

# pytorch also has lots of loss functions available. We can also easily make our own.
# to start, we will use the mean squarred error:
criterion = nn.MSELoss()

And calling the training function:

In [ ]:
sn, (train_loss_dict, val_loss_dict) = train(
    model=sn,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=n_epochs,
    optimiser=optimiser,
    criterion=criterion,
    )

Let's see what the loss looks like:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4)) # plotting area

# plot training loss
ax.plot(
    train_loss_dict['step'], train_loss_dict['loss'],
    label='Train'
    )

# plot training loss
ax.plot(
    val_loss_dict['step'], val_loss_dict['loss'],
    label='Val'
    )

# formatting
ax.set_title('Training Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Steps')
ax.legend()
# show plot
fig.show()

Did the values look like the true equation? Let's see:

In [ ]:
print(f"After training, the single neuron has weights:\n {sn.weight} "\
    f"\n \n and bias:\n {sn.bias}")

In [ ]:
print(f"The true equation has weights:\n {m} "\
    f"\n \n and bias:\n {c}")

The model approximated the function well, and we can see that the validation loss in the graph above went to 0. This tells us that the model didn't overfit.

As you will have seen from the linear models lab, this is essentially a linear regression model and training it in this way is not a good idea, since closed solutions exist! This is simply an example of one of the basic building blocks of a Neural Network. We will now turn our attention to some of the other building blocks of neural networks, and how we can use them to create complex machine learning models.

### Activation Functions

Activation functions are non-linear functions that are placed after a layer of neurons to produce non-linearity in outputs. To visualise this problem, let's try to solve the following classification problem:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.datasets as datasets

Loading some synthetic data:

In [ ]:
X, y = datasets.make_moons(1000, noise=0.15, random_state=random_state.next())

# train-val splits:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size=0.75, random_state=random_state.next()
    )

# scaling the data
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train)).float()
X_val = torch.tensor(scaler.transform(X_val)).float()

y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)

This looks like:

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,4))
ax1, ax2 = axes

ax1.scatter(x=X_train[:,0], y=X_train[:,1], c=y_train, alpha=0.5, cmap=binary_cmap, edgecolor='black')
ax2.scatter(x=X_val[:,0], y=X_val[:,1], c=y_val, alpha=0.5, cmap=binary_cmap, edgecolor='black')

ax1.set_title('Train')
ax2.set_title('Val')

fig.show()

Let us try and use our linear neuron model to try and classify this data:

In [ ]:
# since we won't be using activation functions,
# we need the result to be a float.
# we also need to reshape the data to (n_data_points, 1)
# we will see why later.
y_train_reg = y_train.float().unsqueeze(1)
y_val_reg = y_val.float().unsqueeze(1)

In [ ]:
print(f"The shape of the input array is {X_train.shape} "\
    f"and the shape of the targets is {y_train_reg.shape}")

In [ ]:
sn = nn.Sequential(
    nn.Linear(X_train.shape[1], 100*X_train.shape[1]),
    nn.Linear(100*X_train.shape[1], 100*X_train.shape[1]),
    nn.Linear(100*X_train.shape[1], 1),
)

In [ ]:
# this will only work in the same directory as the file `model_trainer`
from model_trainer import train

In [ ]:
# the number of times to run through the whole dataset
n_epochs = 100
# the batch size for the data
batch_size = 128

# wrapper data in dataset
train_dataset = torch.utils.data.TensorDataset(X_train, y_train_reg)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val_reg)

# put dataset in train loader
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    )
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True,
    )

In [ ]:
# pytorch has lots of optimisers available for use. Here we use Adam:
optimiser = torch.optim.Adam(
    params=sn.parameters(), # need to pass the optimiser the model parameters
    lr=0.001, # and the learning rate
    weight_decay=0.0001
    )

# mean squarred error is used since we are doing regression
criterion = nn.MSELoss()

In [ ]:
sn, (train_loss_dict, val_loss_dict) = train(
    model=sn,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=n_epochs,
    optimiser=optimiser,
    criterion=criterion,
    )

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4)) # plotting area

# plot training loss
ax.plot(
    train_loss_dict['step'], train_loss_dict['loss'],
    label='Train'
    )

# plot training loss
ax.plot(
    val_loss_dict['step'], val_loss_dict['loss'],
    label='Val'
    )

# formatting
ax.set_title('Training Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Steps')
ax.legend()
# show plot
fig.show()

In [ ]:
print(f"A single prediction from the model looks as follows: {sn(X_train[0])}")

The above isn't really a prediction of label, and instead is a regression prediction of the label. Let's see what this decision boundary looks like on a graph:

In [ ]:
# the code in this cell is not important and is written to enable us to 
# plot a decision boundary with a pytorch model
from decision_plotter import pytorch_decision_boundary

class PTtoDB(object):
    def __init__(self, model):
        self.model=model
    
    @torch.no_grad()
    def predict(self, inputs):
        if type(inputs) != torch.Tensor:
            inputs = torch.tensor(inputs).float()
        self.model.eval()
        return self.model(inputs).reshape(-1)

Let us look at the decision boundary. We will take the prediction as being the larger of of the

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,6))
ax = pytorch_decision_boundary(PTtoDB(sn), X=X_val, ax=ax, cmap=binary_cmap)
ax.scatter(x=X_val[:,0], y=X_val[:,1], c=y_val, alpha=0.5, cmap=binary_cmap, edgecolor='black')
ax.set_title('Boundaries on the Test Set')
fig.show()

Now, let's try to add a non-linear decision boundary to the code and predict the labels instead:

In [ ]:
nl = nn.Sequential(
    nn.Linear(X_train.shape[1], 100*X_train.shape[1]),
    nn.ReLU(), # non linear
    nn.Linear(100*X_train.shape[1], 100*X_train.shape[1]),
    nn.ReLU(), # non linear
    nn.Linear(100*X_train.shape[1], 2),
)

In [ ]:
print(f"The shape of the input array is {X_train.shape} "\
    f"and the shape of the targets is {y_train.shape}")

In [ ]:
# this will only work in the same directory as the file `model_trainer`
from model_trainer import train

In [ ]:
# the number of times to run through the whole dataset
n_epochs = 100
# the batch size for the data
batch_size = 128

# wrapper data in dataset
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)

# put dataset in train loader
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    )
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True,
    )

In [ ]:
# pytorch has lots of optimisers available for use. Here we use Adam:
optimiser = torch.optim.Adam(
    params=nl.parameters(), # need to pass the optimiser the model parameters
    lr=0.01, # and the learning rate
    weight_decay=0.0001
    )

# This time we will use cross entropy loss, 
# which includes a non-linear activation function
criterion = nn.CrossEntropyLoss()

In [ ]:
nl, (train_loss_dict, val_loss_dict) = train(
    model=nl,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=n_epochs,
    optimiser=optimiser,
    criterion=criterion,
    )

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4)) # plotting area

# plot training loss
ax.plot(
    train_loss_dict['step'], train_loss_dict['loss'],
    label='Train'
    )

# plot training loss
ax.plot(
    val_loss_dict['step'], val_loss_dict['loss'],
    label='Val'
    )

# formatting
ax.set_title('Training Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Steps')
ax.legend()
# show plot
fig.show()

In [ ]:
print(f"A single prediction from the model "\
    f"looks as follows: {nl(X_train[0].unsqueeze(0))}")

In [ ]:
print(f"These values can easily be turned to probabilities "\
    f"as by using the softmax function:\n {F.softmax(nl(X_train[0].unsqueeze(0)), dim=1, )}")

The above isn't really a prediction of label, and instead is a regression prediction of the label. Let's see what this decision boundary looks like on a graph:

In [ ]:
# the code in this cell is not important and is written to enable us to 
# plot a decision boundary with a pytorch model
from decision_plotter import pytorch_decision_boundary

class PTtoDB(object):
    def __init__(self, model):
        self.model=model
    
    @torch.no_grad()
    def predict(self, inputs):
        if type(inputs) != torch.Tensor:
            inputs = torch.tensor(inputs).float()
        self.model.eval()
        output = self.model(inputs)
        probabilities =  F.softmax(output, dim=1)
        return probabilities[:,1].reshape(-1) # return probability of class 1

Let us look at the decision boundary. We will take the prediction as being the larger of of the

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,6))
ax = pytorch_decision_boundary(PTtoDB(nl), X=X_val, ax=ax, cmap=binary_cmap)
ax.scatter(x=X_val[:,0], y=X_val[:,1], c=y_val, alpha=0.5, cmap=binary_cmap, edgecolor='black')
ax.set_title('Boundaries on the Test Set')
fig.show()

We can see that the decision bounday fits the data much closer! All we did was add a non-linear layer to the model and include loss function that contains a non linear layer, as well as using a non-linear function to turn log-probabilities to probabilities.

Now that we understand the motivation behind using non linear activation functions, let's take a look at some of the most common functions:

In [ ]:
x = torch.linspace(-1,1,100)
y = F.relu(x)

fig, ax = plt.subplots(1,1,figsize=(6,6))
ax.plot(x, y)
ax.grid()
ax.set_ylabel('$y$')
ax.set_xlabel('$x$')
ax.set_title('ReLU Function')
plt.show()

In [ ]:
x = torch.linspace(-1,1,100)
y = F.leaky_relu(x, negative_slope=0.1)

fig, ax = plt.subplots(1,1,figsize=(6,6))
ax.plot(x, y)
ax.grid()
ax.set_ylabel('$y$')
ax.set_xlabel('$x$')
ax.set_title('Leaky ReLU Function')
plt.show()

In [ ]:
x = torch.linspace(-3,3,100)
y = F.tanh(x)

fig, ax = plt.subplots(1,1,figsize=(6,6))
ax.plot(x, y)
ax.grid()
ax.set_ylabel('$y$')
ax.set_xlabel('$x$')
ax.set_title('Tanh Function')
plt.show()

In [ ]:
x = torch.linspace(-8,8,100)
y = F.sigmoid(x)

fig, ax = plt.subplots(1,1,figsize=(6,6))
ax.plot(x, y)
ax.grid()
ax.set_ylabel('$y$')
ax.set_xlabel('$x$')
ax.set_title('Sigmoid Function')
plt.show()

## A Feed Forward Network

Now that we have seen how to access a single layer of neurons and some of the most common activation functions, as well as studying how these models are trained we will now see how more complex networks can be built.

### Basics of Building a Network

Pytorch allows for the user to build complex, feed forward networks easily. Let us see how you might create a multi-layer perceptron:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

The `nn.Sequential` function (documentation [here](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)) allows you to write a sequence with any number of `nn.Module` ([here](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)) objects in order of execution. For example, a model with 3 layers of neurons in order can be written as follows:

In [ ]:
input_size = 10 # example input size
output_size = 2 # example output size

# a 3 layer network mapping 10 features -> 2 features
ffn = nn.Sequential(
    nn.Linear(input_size, 20), # (layer_input_size, layer_output_size)
    nn.Linear(20, 20),
    nn.Linear(20, output_size),
    )

We can view the network by printing it:

In [ ]:
print(ffn)

Adding non-linear layers is also simple, we simply add those to the `nn.Sequential` function:

In [ ]:
input_size = 10 # example input size
output_size = 2 # example output size

# a 3 layer network mapping 10 features -> 2 features
nn.Sequential(
    nn.Linear(input_size, 20),
    nn.ReLU(), # relu function added
    nn.Linear(20, 20),
    nn.Tanh(), # tanh function added
    nn.Linear(20, output_size),
    nn.LeakyReLU(), # leaky relu added
    )

Similarly, to create more complicated models, like transformers, this can be as simple as:

Transformers are the backbone of many incredible deep learning models such as [ChatGPT](https://openai.com/blog/chatgpt/) which is worth testing out!

In [ ]:
# example transformer model 
nn.Sequential(
    nn.Linear(1000, 512),
    # transformers need a positional encoding layer here, see:
    #https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    # to learn how to add one.
    nn.Transformer(512, batch_first=True), 
    )

As we can see, this is a much more complicated model!

If we want to build custom models or blocks, we can do the following using python classes and sub-classing the `nn.Module` class:

In [ ]:
class FFBlock(nn.Module):
    def __init__(self, input_size, output_size):
        # always needs to be the first thing done when building a custom model
        super(FFBlock, self).__init__() 
        # defining a custom block
        self.net = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.ReLU(),
            nn.BatchNorm1d(output_size), # can even add batch norm like this
            nn.Dropout(p=0.2) # dropout with rate = 0.2
            )
    def forward(self, inputs):
        return self.net(inputs)

Now we can use this class in other models too:

In [ ]:
larger_ff_model = nn.Sequential(
    FFBlock(10, 100),
    FFBlock(100, 100),
    FFBlock(100, 100),
    nn.Linear(100, 2)
    )

This produces the model:

In [ ]:
larger_ff_model

All of these models can be used in regression or classification tasks. In general:

- **Regression**: 
    - Ensure that the feature output size is the same as the output size of the targets.
    - Use a loss function designed for regression such as `nn.MSELoss`.
    - **Example**: Predicting the temperature tomorrow based on today's data.

- **Classification**:
    - Ensure that the feature output size is the same as the number of classes being predicted. For example, if you're performning binary classification output two features, etc. The outputs can be interpretted as the log-probability of the input being from the coressponding class. To get the actual probabilities, you can use `torch.nn.functional.softmax(outputs, dim=1)`. 
    - Use a loss function designed for classification such as `nn.CrossEntropyLoss`.
    - For multi-label classification (each data point can have multiple labels), you may want to use `nn.BCEWithLogitsLoss()`.
    - **Example**: Predicting whether it will rain or not tomorrow, or if doing multi-label classification, predicting whether it will rain tomorrow and/or the professor wears a tie to lectures tomorrow (since these are likely to be mutually exclusive).

The following are examples of classification and regression:

In [ ]:
# regression - 3 linear layers mapping 10 input features to 1 output feature
r_model = nn.Sequential(nn.Linear(10,10), nn.Linear(10,10), nn.Linear(10,1))
criterion = nn.MSELoss()
optimiser = torch.optim.Adam(r_model.parameters(), lr=0.01)

# classification - 3 linear layers mapping 10 input features to 5 classes
c_model = nn.Sequential(nn.Linear(10,10), nn.Linear(10,10), nn.Linear(10,5))
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(c_model.parameters(), lr=0.01)

# multi-label classification - 3 linear layers mapping 10 input features to 5 attributes
mc_model = nn.Sequential(nn.Linear(10,10), nn.Linear(10,10), nn.Linear(10,5))
criterion = nn.BCEWithLogitsLoss()
optimiser = torch.optim.Adam(mc_model.parameters(), lr=0.01)

### Maybe an Easier Way

If all you need is a multilayer perceptron (linear layers combined with drop out and non linear functions), then sklearn can provide this. See [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) and [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html).

In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor

In [ ]:
model = MLPClassifier()
model.get_params()

In [ ]:
model = MLPRegressor()
model.get_params()

These models are great if you don't need anything custom and just want to have a simple network perform a classification or regression task. For example, let's look at the task we gave to our neural networks earlier:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.datasets as datasets

Loading some synthetic data. These do not need to be tensors, since sklearn works on numpy arrays:

In [ ]:
X, y = datasets.make_moons(1000, noise=0.15, random_state=random_state.next())

# train-val splits:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size=0.75, random_state=random_state.next()
    )

# scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

This looks like:

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,4))
ax1, ax2 = axes

ax1.scatter(x=X_train[:,0], y=X_train[:,1], c=y_train, alpha=0.5, cmap=binary_cmap, edgecolor='black')
ax2.scatter(x=X_val[:,0], y=X_val[:,1], c=y_val, alpha=0.5, cmap=binary_cmap, edgecolor='black')

ax1.set_title('Train')
ax2.set_title('Val')

fig.show()

In [ ]:
# same as our pytorch model, but comes with extra performance options as default
mlp = MLPClassifier(
    hidden_layer_sizes=[
        100*X_train.shape[1],
        100*X_train.shape[1],
        ],
    activation='relu',
    solver='adam',
    batch_size=128,
    learning_rate='constant',
    learning_rate_init=0.01,
    random_state=random_state.next()
    )

In [ ]:
mlp.fit(X_train, y_train)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,6))

dbd = DecisionBoundaryDisplay.from_estimator(
    estimator=mlp,
    X=X_val,
    grid_resolution=200,
    plot_method='contourf',
    response_method='predict_proba',
    ax=ax,
    cmap=binary_cmap,
    alpha=0.5,
    eps=0.3,
    levels=100,
    )

ax.scatter(x=X_val[:,0], y=X_val[:,1], c=y_val, alpha=0.5, cmap=binary_cmap, edgecolor='black')
ax.set_title('Sklearn MLP')
fig.show()

As we can see, this was considerably easier to implement, and the `MLPClassifier` and `MLPRegressor` classes give you quick access to a wide variety of interesting options.

### An Example

Let's see how a feed forward network can be used to predict the [fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). This is a simple dataset containing images of clothes. We will flatten these images so that it is possible to make predictions using a feed forward MLP. We will then compare the pytorch and sklearn implementations.

In [ ]:
import torchvision # to get data
import torchvision.transforms as transforms

from dataset import MemoryDataset

In [ ]:
transform_images = transforms.Compose([
                        transforms.PILToTensor(),
                        transforms.ConvertImageDtype(torch.float),
                        transforms.Normalize(mean=0, std=1),
                        nn.Flatten(start_dim=0),
                        ])

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data/',
    train=True,
    download=True, 
    transform=transform_images
    )

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data/',
    train=False,
    download=True, 
    transform=transform_images
    )

# keeps data in memory after loading
train_dataset = MemoryDataset(train_dataset, now=False,)
test_dataset = MemoryDataset(test_dataset, now=False,)

A few example images:

In [ ]:
images_to_show = [train_dataset[i][0].reshape(28,28) for i in range(5)]

fig, axes = plt.subplots(1,5,figsize=(15,3))

for nax, ax in enumerate(axes):
    ax.imshow(
        images_to_show[nax].numpy(),
        cmap='Greys'
        )

Let us try and train a pytorch model:

In [ ]:
# the training function we made
from model_trainer import train

In [ ]:
# setting up the data loaders
# splitting the data into train and val
n_train_set, n_val_set = [
    int(0.75*len(train_dataset)), len(train_dataset)-int(0.75*len(train_dataset))
    ]

train_dataset_split, val_dataset_split = torch.utils.data.random_split(
    train_dataset, lengths=[n_train_set, n_val_set],
    )

# the number of times to run through the whole dataset
n_epochs = 10
# the batch size for the data
batch_size = 512

# put datasets in train loader
train_loader = torch.utils.data.DataLoader(
    train_dataset_split, batch_size=batch_size, shuffle=True,
    )
val_loader = torch.utils.data.DataLoader(
    val_dataset_split, batch_size=batch_size, shuffle=True,
    )
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False,
    )

# model
mlp = nn.Sequential(
    nn.Linear(784, 16),
    #nn.Dropout(0.2), # dropout can be easily added like this
    nn.ReLU(),
    nn.Linear(16, 16), 
    #nn.Dropout(0.2), # dropout can be easily added like this
    nn.ReLU(),
    nn.Linear(16,10),
    )

optimiser = torch.optim.Adam(mlp.parameters(), lr=0.01, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()

And finally, we will use our function to train the model:

In [ ]:
# train the model
model, (train_loss_dict, val_loss_dict) = train(
    model=mlp,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=n_epochs,
    optimiser=optimiser,
    criterion=criterion
    )

This gives us the loss plot:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4)) # plotting area

# plot training loss
ax.plot(
    train_loss_dict['step'], train_loss_dict['loss'],
    label='Train'
    )

# plot training loss
ax.plot(
    val_loss_dict['step'], val_loss_dict['loss'],
    label='Val'
    )

# formatting
ax.set_title('Training Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Steps')
ax.legend()
# show plot
fig.show()

To calculate the accuracy, we need to count the number of correct instances over the test loader:

Similarly to our training function, we can define a predictions function that can return the predictions, true labels, and the raw outputs:

In [ ]:
@torch.no_grad() # no gradients to calculate
def predict(
    model:nn.Module, 
    test_loader:torch.utils.data.DataLoader,
    ):
    '''
    This function will iterate over the :code:`test_loader`
    and return the outputs of the model
    applied to the data.

    Arguments
    ---------

    - model: nn.Module:
        The model used to make predictions.

    - test_loader: torch.utils.data.DataLoader:
        The data to predict on.
    
    Returns
    ---------

    - outputs: torch.Tensor:
        The outputs of the model
    
    - labels: torch.Tensor:
        The ground truth labels collected
        from :code:`test_loader`.

    '''
    
    # lists to contain the output data
    targets = []
    outputs = []

    # adding model to GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()

    # iterating over the test_loader with a progress bar
    for input, target in tqdm.tqdm(test_loader, desc='Predicting'):
        # ==== push data to GPU if available ====
        input = input.to(device)
        # ==== forward pass ====
        output = model(input)
        # ==== saving outputs and labels ====
        outputs.append(output.cpu())
        targets.append(target) # target was never pushed to GPU so remains on cpu
    
    # turning outputs into torch tensors instead of lists
    outputs = torch.cat(outputs)
    targets = torch.cat(targets)

    model.to('cpu') # return the model to the CPU

    return outputs, targets

We will save the above function in the file `model_predictor.py` so that we can import it easily later.

In [ ]:
test_mlp_predictions, test_labels = predict(model, test_loader=test_loader)
# finding the class of max probability
_, test_mlp_predictions = torch.max(test_mlp_predictions, dim=1) 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_mlp_accuracy = accuracy_score(test_dataset.targets.numpy(), test_mlp_predictions.numpy())

In [ ]:
print(f"The accuracy of the pytorch MLP is {test_mlp_accuracy*100:.2f}%")

Now, let's see how well an sklearn MLP does:

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

Now we need to make numpy arrays from the same data used in the pytorch training:

In [ ]:
# since the data originated from pytorch dataloaders, we need to turn these into numpy 
# arrays for our sklearn model

X_train, y_train = [], []
for inputs, targets in train_loader:
    X_train.append(inputs.numpy())
    y_train.append(targets.numpy())

X_val, y_val = [], []
for inputs, targets in val_loader:
    X_val.append(inputs.numpy())
    y_val.append(targets.numpy())

X_test, y_test = [], []
for inputs, targets in test_loader:
    X_test.append(inputs.numpy())
    y_test.append(targets.numpy())

X_train = np.vstack(X_train)
y_train = np.concatenate(y_train)
X_val = np.vstack(X_val)
y_val = np.concatenate(y_val)
X_test = np.vstack(X_test)
y_test = np.concatenate(y_test)

In [ ]:
# same as our pytorch model, but comes with extra performance options as default
# also MLPClassifier does not support dropout
mlp = MLPClassifier(
    hidden_layer_sizes=[
        16,
        16,        
        ],
    activation='relu',
    solver='adam',
    batch_size=512,
    learning_rate='constant',
    learning_rate_init=0.01,
    random_state=random_state.next()
    )

In [ ]:
mlp.fit(X_train, y_train)

In [ ]:
test_mlp_sk_predictions = mlp.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_mlp_sk_accuracy = accuracy_score(y_test, test_mlp_sk_predictions)

In [ ]:
print(f"The accuracy of the pytorch MLP is {test_mlp_sk_accuracy*100:.2f}%")

Here, Sklearn's model was arguably easier to implement, and performed better. However, when more complex datasets or models are being experimented with, Pytorch is required.

Also, pytorch allows you to keep all datasets on disk and ensures that large datasets do not cause memory problems. Sklearn prefers data to be in memory. This is why it ran faster here, since converting the data from pytorch datasets to numpy arrays loaded the data into memory.

## More Complex Neural Networks

**The below will take a long time to run, and you might find it more helpful to view the version of this notebook that is already run**

We will now look at an example of training a complex model on ECG data, located online. This will also allow us to provide an example on how to build custom datasets from online data.

These more complicated examples require pytorch models and their flexibility:

In [ ]:
# this will only work in the same directory as the file `model_trainer`
from model_trainer import train

Within the `dataset.py` file in this directory, we have provided an example of loading a complex ECG dataset from online ([here, PTB-XL](https://physionet.org/content/ptb-xl/1.0.0/)) and loading it in a pytorch dataset.

In [ ]:
from dataset import PTB_XL, MemoryDataset

**NOTE:** The following code will download the ECG dataset and unpack it. This is quite a large dataset and might not work on google colab.

In [ ]:
train_dataset = PTB_XL(
    data_path='./data/',
    train=True,
    sampling_rate=100,
    binary=True,
    )

test_dataset = PTB_XL(
    data_path='./data/',
    train=True,
    sampling_rate=100,
    binary=True,
    )

feature_names = train_dataset.feature_names

What does this data look like?

In [ ]:
idx_normal = np.argmax(train_dataset.targets == 0) # first normal ecg
idx_abnormal = np.argmax(train_dataset.targets == 1) # first abnormal ecg

In [ ]:
inputs, _ = train_dataset[idx_normal]

fig, axes = plt.subplots(3, inputs.shape[0]//3, figsize=(12,8))

# getting the colours to make the plot look nicer!
colors = plt.rcParams["axes.prop_cycle"]()

for nax, ax in enumerate(np.ravel(axes)):
    ax.plot(inputs[nax,:], color=next(colors)["color"])
    ax.set_title(f'Feature {feature_names[nax]}')

fig.suptitle('Features for Normal ECG Result')
fig.subplots_adjust(hspace=0.3, wspace=0.3)
fig.show()

In [ ]:
inputs, _ = train_dataset[idx_abnormal]

fig, axes = plt.subplots(3, inputs.shape[0]//3, figsize=(12,8))

# getting the colours to make the plot look nicer!
colors = plt.rcParams["axes.prop_cycle"]()

for nax, ax in enumerate(np.ravel(axes)):
    ax.plot(inputs[nax,:], color=next(colors)["color"])
    ax.set_title(f'Feature {feature_names[nax]}')

fig.suptitle('Features for Abnormal ECG Result')
fig.subplots_adjust(hspace=0.3, wspace=0.3)
fig.show()

Within the `dataset.py` file, we have also given an example of a wrapper that can be built for a dataset to do interesting things! We have included a wrapper that loads all of the dataset into memory after the first epoch:

In [ ]:
train_dataset = MemoryDataset(train_dataset, now=False)
test_dataset = MemoryDataset(test_dataset, now=False)

We now load a 1D ResNet model that will be used to train a model on this dataset. This is a complicated model, with skip connections and many layers. It is an example of how flexible pytorch can be:

To view the code for this model, feel free to look through the file `resnet.py`. 

In [ ]:
# importing a ResNet model from the file resnet.py
from resnet import ResNet

Now, let's train this ResNet model to make predictions on an ECG dataset:

In [ ]:
# setting up the data loaders
# splitting the data into train and val
n_train_set, n_val_set = [
    int(0.75*len(train_dataset)), len(train_dataset)-int(0.75*len(train_dataset))
    ]

train_dataset_split, val_dataset_split = torch.utils.data.random_split(
    train_dataset, lengths=[n_train_set, n_val_set],
    )

# the number of times to run through the whole dataset
n_epochs = 15
# the batch size for the data
batch_size = 128

# put datasets in train loader
train_loader = torch.utils.data.DataLoader(
    train_dataset_split, batch_size=batch_size, shuffle=True,
    )
val_loader = torch.utils.data.DataLoader(
    val_dataset_split, batch_size=batch_size, shuffle=True,
    )
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False,
    )

In [ ]:
resnet = ResNet(
    input_dim=1000,
    input_channels=12,
    kernel_size=15,
    n_output=2,
    dropout_rate=0.2,
    )

optimiser = torch.optim.Adam(
    params=resnet.parameters(), 
    lr=0.001, 
    weight_decay=0.0001, 
    betas=(0.99, 0.999),
    )
criterion = nn.CrossEntropyLoss()

In [ ]:
# train the model
resnet, (train_loss_dict, val_loss_dict) = train(
    model=resnet,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=n_epochs,
    optimiser=optimiser,
    criterion=criterion
    )

This gives us the loss plot:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4)) # plotting area

# plot training loss
ax.plot(
    train_loss_dict['step'], train_loss_dict['loss'],
    label='Train'
    )

# plot training loss
ax.plot(
    val_loss_dict['step'], val_loss_dict['loss'],
    label='Val'
    )

# formatting
ax.set_title('Training Loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Steps')
ax.legend()
# show plot
fig.show()

In [ ]:
# import the function we defined earlier for 
# making predictions on a test data loader
from model_predictor import predict

In [ ]:
test_probabilities, test_targets = predict(resnet, test_loader=test_loader)
# finding the class of max probability
_, test_predictions = torch.max(test_probabilities, dim=1) 
test_probabilities = test_probabilities[:,1]

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
test_resnet_accuracy = accuracy_score(
    test_targets.numpy(), test_predictions.numpy()
    )

In [ ]:
print(f"The accuracy of the pytorch ResNet is {test_resnet_accuracy*100:.2f}%")

In [ ]:
test_resnet_auc_roc = roc_auc_score(
    test_targets.numpy(), test_probabilities.numpy()
    )

In [ ]:
print(f"The area under the ROC of the pytorch ResNet is {test_resnet_auc_roc*100:.2f}%")

Here, we trained a model to classify ECG results as normal or abnormal, and were able to get good accuracy and area under the ROC.